# 在 LimaIndex 中使用 Ragas

## 准备

In [15]:
%%time
%%capture

!pip install ragas

CPU times: user 11.9 ms, sys: 4.92 ms, total: 16.8 ms
Wall time: 3.85 s


In [16]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "颐和园",
]

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 3.06 s, sys: 379 ms, total: 3.44 s
Wall time: 3.2 s


1

In [23]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings

Settings.llm =OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size=128
Settings.chunk_overlap=10

evaluating_llm=OpenAILike(
    model='gpt-4-turbo', 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.0,
    request_timeout=60.0
)

CPU times: user 396 µs, sys: 0 ns, total: 396 µs
Wall time: 399 µs


In [18]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.23 ms, sys: 142 µs, total: 1.38 ms
Wall time: 1.32 ms


In [19]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

CPU times: user 933 ms, sys: 56.8 ms, total: 989 ms
Wall time: 20.1 s


## 评估响应的忠实度

In [24]:
%%time

from llama_index.core.evaluation import FaithfulnessEvaluator

response = query_engine.query(
    "颐和园在哪里?"
)

evaluator = FaithfulnessEvaluator(llm=evaluating_llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result

CPU times: user 77.4 ms, sys: 0 ns, total: 77.4 ms
Wall time: 2.44 s


EvaluationResult(query=None, contexts=['收藏\n查看我的收藏\n0有用+1\n颐和园，中国清朝时期皇家园林，前身为清漪园，坐落在北京西郊，距城区15千米，全园占地3.009平方千米（其中颐和园世界文化遗产区面积为2.97平方千米），水面约占四分之三。', '光绪十四年（1888年）重建，改称颐和园，作消夏游乐地。光绪二十六年（1900年），颐和园又遭“八国联军”的破坏，珍宝被劫掠一空。清朝灭亡后，颐和园在军阀混战和国民党统治时期，又遭破坏。'], response='颐和园坐落在北京西郊。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [30]:
response.response

'颐和园坐落在北京西郊。'

In [25]:
# 是否符合检索到的上下文，即是否不存在幻觉

eval_result.passing

True

In [26]:
eval_result.score

1.0

In [27]:
eval_result.feedback

'YES'

In [29]:
%%time

response_str = response.response
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        response=response_str, contexts=[source_node.get_content()]
    )
    print(str(eval_result.passing))

True
False
CPU times: user 17.5 ms, sys: 3.78 ms, total: 21.3 ms
Wall time: 3.31 s


## 评估查询和响应的相关度

In [31]:
%%time

from llama_index.core.evaluation import RelevancyEvaluator

evaluator = RelevancyEvaluator(llm=evaluating_llm)

query = "颐和园在哪里?"
response = query_engine.query(query)

eval_result = evaluator.evaluate_response(query=query, response=response)
eval_result

CPU times: user 28.2 ms, sys: 370 µs, total: 28.5 ms
Wall time: 4.37 s


EvaluationResult(query='颐和园在哪里?', contexts=['收藏\n查看我的收藏\n0有用+1\n颐和园，中国清朝时期皇家园林，前身为清漪园，坐落在北京西郊，距城区15千米，全园占地3.009平方千米（其中颐和园世界文化遗产区面积为2.97平方千米），水面约占四分之三。', '光绪十四年（1888年）重建，改称颐和园，作消夏游乐地。光绪二十六年（1900年），颐和园又遭“八国联军”的破坏，珍宝被劫掠一空。清朝灭亡后，颐和园在军阀混战和国民党统治时期，又遭破坏。'], response='颐和园坐落在北京西郊。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [32]:
eval_result.score

1.0

## 自动生成问题

In [35]:
%%time

from llama_index.core.llama_dataset.generator import RagDatasetGenerator

dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=evaluating_llm,
    num_questions_per_chunk=10,  # set the number of questions per nodes
)

rag_dataset = dataset_generator.generate_questions_from_nodes()
questions = [e.query for e in rag_dataset.examples]

CPU times: user 2.53 s, sys: 37.3 ms, total: 2.57 s
Wall time: 8min 22s


In [37]:
questions[:10]

['What was the original name of the Summer Palace before it was called "颐和园"?',
 'Where is the Summer Palace located in relation to the city center of Beijing?',
 'Approximately how far is the Summer Palace from the urban area of Beijing?',
 'What is the total area of the Summer Palace in square kilometers?',
 "How much of the Summer Palace's total area is classified as a World Heritage site?",
 "What percentage of the Summer Palace's area is covered by water?",
 'During which Chinese dynasty was the Summer Palace built?',
 'What is the significance of the Summer Palace in the context of Chinese royal gardens?',
 'Calculate the area of the Summer Palace that is not included in the World Heritage site.',
 "Based on the description, identify whether the Summer Palace's location is known for its urban connectivity or secluded nature."]

In [38]:
len(questions)

2002